# MoleculeACE - ChEMBL cliff evaluation

Finally, once our models are trained on MoleculeACE data, we evaluate them on the accompanying MoleculeACE test data.

* ChEMBL234 - Dopamine D3 receptor
* ChEMBL4203 - Dual specificity protein kinase
* ChEMBL2047 - Farnesoid X receptor
* ChEMBL4616 - Ghrelin receptor
* ChEMBL264 - Histamine H3 receptor
* ChEMBL2835 - Janus kinase 1
* ChEMBL4792 - Orexin receptor 2

## Setup

### Colab mount

In [1]:
import os.path

try:
    from google.colab import drive
    drive.mount('/content/drive')
    _home = 'drive/MyDrive/tlacamr'
except ImportError:
    _home = '~'
finally:
    project_root = os.path.join(_home, 'tlacamr')

print(project_root)
%cd $project_root

Mounted at /content/drive
drive/MyDrive/tlacamr/tlacamr
/content/drive/MyDrive/tlacamr/tlacamr


### Package install

In [2]:
%%capture
!pip install .
##once repo is public: !pip install git+https://github.com/cmvcordova/my-repo


### Imports

#### Main evaluation function

In [3]:
## general
import torch
import numpy as np
import os
import logging
import glob
import csv
from lightning import Trainer
from torch.utils.data import DataLoader

## acsuite imports
#from src.models.encoding_module import ACAModule
from src.models.property_prediction_module import ACAPPModule
from src.data.property_prediction.moleculeace_dataset import MoleculeACEDataset
from src.data.property_prediction.moleculeace_datamodule import MoleculeACEDataModule
## molfeat related
import datamol as dm
from molfeat.calc import FPCalculator
from molfeat.trans import MoleculeTransformer

## metrics
from MoleculeACE import calc_rmse, calc_cliff_rmse
from sklearn.metrics import roc_auc_score, mean_squared_error

pretrained_model_dir = os.path.join("src", "models", "pretrained", "property_prediction")

def evaluate_datasets(datasets,
                      model_type=None,
                      fingerprint_length=2048,
                      ):
    results = {}

    for dataset_info in datasets:
        dataset_name = dataset_info["name"]
        task = dataset_info["task"]

        task_key = f"{dataset_name}_{task}"
        molfeat_featurizer = MoleculeTransformer(FPCalculator('ecfp', length=fingerprint_length, radius=4))
        # Instantiate MoleculeACEDataset for the test split
        test_dataset = MoleculeACEDataset(dataset_name=dataset_name, data_split='test', task=task, molfeat_featurizer=molfeat_featurizer)
        # Create DataLoader for the test split
        test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False)

        # Find the correct checkpoint path using glob
        # Adjust the path to include the model_type directory and wildcard pattern for subdirectories
        checkpoint_pattern = os.path.join(pretrained_model_dir, task, model_type, dataset_name, "checkpoints", "epoch=*.ckpt")
        checkpoint_files = glob.glob(checkpoint_pattern)
        checkpoint_files.sort()  # Sort the files to get the latest
        if checkpoint_files:
            checkpoint_path = checkpoint_files[-1]  # Select the last checkpoint, which is the latest
            model = ACAPPModule.load_from_checkpoint(checkpoint_path)  # Load the model from the checkpoint
            print(f"Loaded checkpoint from {checkpoint_path}")
            print(f"Evaluating {dataset_name} ({task})")

            trainer = Trainer()

            # Perform prediction using the DataLoader
            predictions = trainer.predict(model, dataloaders=test_dataloader)
            y_pred = torch.cat(predictions).numpy()

            # Extract true labels directly from the dataset
            y_true = np.array(test_dataset.y)

            # Evaluate the model
            if task == "regression":
                test_cliffs = MoleculeACEDataset(dataset_name=dataset_name, data_split='test', task='classification').y
                rmse = calc_rmse(y_true, y_pred)
                cliff_rmse = calc_cliff_rmse(y_test_pred=y_pred,
                                             y_test=y_true,
                                             cliff_mols_test=test_cliffs)
                results[task_key] = {"RMSE": rmse, "Cliff_RMSE": cliff_rmse}
            elif task == "classification":
                if y_pred.ndim > 1 and y_pred.shape[1] > 1:
                    # Use the second column for AUROC calculation
                    y_pred_positive_class = y_pred[:, 1]
                else:
                    # Directly use y_pred if it's already the probability of the positive class
                    y_pred_positive_class = y_pred.flatten()

                auroc = roc_auc_score(y_true, y_pred_positive_class)
                results[task_key] = {"AUROC": auroc}

            print(f"Evaluation completed for {dataset_name} ({task})")
        else:
            print(f"No checkpoint found for {dataset_name} ({task})")
            continue
    return results

#### Print, utils

In [4]:
def write_results_to_csv(results_dict, file_path):
    headers = ['Model', 'Dataset', 'Task', 'Metric', 'Value']
    mode = 'w' if not os.path.exists(file_path) else 'a'
    already_exists = os.path.exists(file_path)

    with open(file_path, mode, newline='') as file:
        writer = csv.writer(file)
        if not already_exists:
            writer.writerow(headers)  # Write headers only if the file did not exist

        for model_name, results in results_dict.items():
            for task_key, metrics in results.items():
                dataset, task = task_key.rsplit('_', 1)
                for metric, value in metrics.items():
                    writer.writerow([model_name, dataset, task, metric, value])

## Evaluation

### Declare eval datasets

In [5]:
# Declare datasets. Analysis categories in comments.

chembl_datasets = [
    # DATASET SIZE:
    ## Highest amount of molecules
    {"name": "CHEMBL234_Ki", "task": "classification"},
    {"name": "CHEMBL234_Ki", "task": "regression"},
    ## Lowest amount of molecules
    {"name": "CHEMBL2835_Ki", "task": "classification"},
    {"name": "CHEMBL2835_Ki", "task": "regression"},
    # AMOUNT OF CLIFF PARTNERS:
    ## Highest amount
    {"name": "CHEMBL4203_Ki", "task": "classification"},
    {"name": "CHEMBL4203_Ki", "task": "regression"},
    ## Lowest amount
    {"name": "CHEMBL4616_EC50", "task": "classification"},
    {"name": "CHEMBL4616_EC50", "task": "regression"},
    # HIGHEST SIMILARITY ACROSS METRICS:
    ## Highest mean scaffold similarity
    {"name": "CHEMBL264_Ki", "task": "classification"},
    {"name": "CHEMBL264_Ki", "task": "regression"},
    ## Highest substructure similarity
    {"name": "CHEMBL4792_Ki", "task": "classification"},
    {"name": "CHEMBL4792_Ki", "task": "regression"},
    ## Highest SMILES similarity
    {"name": "CHEMBL2047_EC50", "task": "classification"},
    {"name": "CHEMBL2047_EC50", "task": "regression"}
]

### MLP 2048

In [6]:
MLP_2048_results = evaluate_datasets(chembl_datasets, "MLP_2048", fingerprint_length=2048)

/usr/local/lib/python3.10/dist-packages/lightning/pytorch/utilities/parsing.py:199: Attribute 'net' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['net'])`.


Loaded checkpoint from src/models/pretrained/property_prediction/classification/MLP_2048/CHEMBL234_Ki/checkpoints/epoch=3-step=148.ckpt
Evaluating CHEMBL234_Ki (classification)


INFO: GPU available: False, used: False
INFO:lightning.pytorch.utilities.rank_zero:GPU available: False, used: False
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


Predicting: |          | 0/? [00:00<?, ?it/s]

Evaluation completed for CHEMBL234_Ki (classification)


...

Evaluation completed for CHEMBL2047_EC50 (classification)


/usr/local/lib/python3.10/dist-packages/lightning/pytorch/utilities/parsing.py:199: Attribute 'net' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['net'])`.
INFO: GPU available: False, used: False
INFO:lightning.pytorch.utilities.rank_zero:GPU available: False, used: False
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


Loaded checkpoint from src/models/pretrained/property_prediction/regression/MLP_2048/CHEMBL2047_EC50/checkpoints/epoch=18-step=133.ckpt
Evaluating CHEMBL2047_EC50 (regression)


Predicting: |          | 0/? [00:00<?, ?it/s]

Evaluation completed for CHEMBL2047_EC50 (regression)


### MLP 1024

In [7]:
MLP_1024_results = evaluate_datasets(chembl_datasets, "MLP_1024", fingerprint_length=1024)

/usr/local/lib/python3.10/dist-packages/lightning/pytorch/utilities/parsing.py:199: Attribute 'net' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['net'])`.
INFO: GPU available: False, used: False
INFO:lightning.pytorch.utilities.rank_zero:GPU available: False, used: False
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


Loaded checkpoint from src/models/pretrained/property_prediction/classification/MLP_1024/CHEMBL234_Ki/checkpoints/epoch=4-step=185.ckpt
Evaluating CHEMBL234_Ki (classification)


Predicting: |          | 0/? [00:00<?, ?it/s]

Evaluation completed for CHEMBL234_Ki (classification)


/usr/local/lib/python3.10/dist-packages/lightning/pytorch/utilities/parsing.py:199: Attribute 'net' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['net'])`.
INFO: GPU available: False, used: False
INFO:lightning.pytorch.utilities.rank_zero:GPU available: False, used: False
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


...

Evaluation completed for CHEMBL2047_EC50 (classification)


/usr/local/lib/python3.10/dist-packages/lightning/pytorch/utilities/parsing.py:199: Attribute 'net' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['net'])`.
INFO: GPU available: False, used: False
INFO:lightning.pytorch.utilities.rank_zero:GPU available: False, used: False
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


Loaded checkpoint from src/models/pretrained/property_prediction/regression/MLP_1024/CHEMBL2047_EC50/checkpoints/epoch=9-step=70.ckpt
Evaluating CHEMBL2047_EC50 (regression)


Predicting: |          | 0/? [00:00<?, ?it/s]

Evaluation completed for CHEMBL2047_EC50 (regression)


### MLP 256

In [8]:
MLP_256_results = evaluate_datasets(chembl_datasets, "MLP_256", fingerprint_length=256)

/usr/local/lib/python3.10/dist-packages/lightning/pytorch/utilities/parsing.py:199: Attribute 'net' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['net'])`.
INFO: GPU available: False, used: False
INFO:lightning.pytorch.utilities.rank_zero:GPU available: False, used: False
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


Loaded checkpoint from src/models/pretrained/property_prediction/classification/MLP_256/CHEMBL234_Ki/checkpoints/epoch=25-step=962.ckpt
Evaluating CHEMBL234_Ki (classification)


Predicting: |          | 0/? [00:00<?, ?it/s]

Evaluation completed for CHEMBL234_Ki (classification)


/usr/local/lib/python3.10/dist-packages/lightning/pytorch/utilities/parsing.py:199: Attribute 'net' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['net'])`.
INFO: GPU available: False, used: False
INFO:lightning.pytorch.utilities.rank_zero:GPU available: False, used: False
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


...

Evaluation completed for CHEMBL2047_EC50 (classification)


/usr/local/lib/python3.10/dist-packages/lightning/pytorch/utilities/parsing.py:199: Attribute 'net' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['net'])`.
INFO: GPU available: False, used: False
INFO:lightning.pytorch.utilities.rank_zero:GPU available: False, used: False
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


Loaded checkpoint from src/models/pretrained/property_prediction/regression/MLP_256/CHEMBL2047_EC50/checkpoints/epoch=9-step=70.ckpt
Evaluating CHEMBL2047_EC50 (regression)


Predicting: |          | 0/? [00:00<?, ?it/s]

Evaluation completed for CHEMBL2047_EC50 (regression)


### MLP_2048_halfstep

In [9]:
halfstepMLP_2048_results = evaluate_datasets(chembl_datasets, "halfstepMLP_2048", fingerprint_length=2048)

/usr/local/lib/python3.10/dist-packages/lightning/pytorch/utilities/parsing.py:199: Attribute 'net' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['net'])`.
INFO: GPU available: False, used: False
INFO:lightning.pytorch.utilities.rank_zero:GPU available: False, used: False
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


Loaded checkpoint from src/models/pretrained/property_prediction/classification/halfstepMLP_2048/CHEMBL234_Ki/checkpoints/epoch=28-step=1073.ckpt
Evaluating CHEMBL234_Ki (classification)


Predicting: |          | 0/? [00:00<?, ?it/s]

Evaluation completed for CHEMBL234_Ki (classification)


/usr/local/lib/python3.10/dist-packages/lightning/pytorch/utilities/parsing.py:199: Attribute 'net' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['net'])`.
INFO: GPU available: False, used: False
INFO:lightning.pytorch.utilities.rank_zero:GPU available: False, used: False
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


...

Evaluation completed for CHEMBL2047_EC50 (classification)


/usr/local/lib/python3.10/dist-packages/lightning/pytorch/utilities/parsing.py:199: Attribute 'net' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['net'])`.
INFO: GPU available: False, used: False
INFO:lightning.pytorch.utilities.rank_zero:GPU available: False, used: False
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


Loaded checkpoint from src/models/pretrained/property_prediction/regression/halfstepMLP_2048/CHEMBL2047_EC50/checkpoints/epoch=8-step=63.ckpt
Evaluating CHEMBL2047_EC50 (regression)


Predicting: |          | 0/? [00:00<?, ?it/s]

Evaluation completed for CHEMBL2047_EC50 (regression)


### MLP_1024_halfstep

In [10]:
halfstepMLP_1024_results = evaluate_datasets(chembl_datasets, "halfstepMLP_1024", fingerprint_length=1024)

/usr/local/lib/python3.10/dist-packages/lightning/pytorch/utilities/parsing.py:199: Attribute 'net' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['net'])`.
INFO: GPU available: False, used: False
INFO:lightning.pytorch.utilities.rank_zero:GPU available: False, used: False
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


Loaded checkpoint from src/models/pretrained/property_prediction/classification/halfstepMLP_1024/CHEMBL234_Ki/checkpoints/epoch=25-step=962.ckpt
Evaluating CHEMBL234_Ki (classification)


Predicting: |          | 0/? [00:00<?, ?it/s]

Evaluation completed for CHEMBL234_Ki (classification)


/usr/local/lib/python3.10/dist-packages/lightning/pytorch/utilities/parsing.py:199: Attribute 'net' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['net'])`.
INFO: GPU available: False, used: False
INFO:lightning.pytorch.utilities.rank_zero:GPU available: False, used: False
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


...

Evaluation completed for CHEMBL2047_EC50 (classification)


/usr/local/lib/python3.10/dist-packages/lightning/pytorch/utilities/parsing.py:199: Attribute 'net' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['net'])`.
INFO: GPU available: False, used: False
INFO:lightning.pytorch.utilities.rank_zero:GPU available: False, used: False
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


Loaded checkpoint from src/models/pretrained/property_prediction/regression/halfstepMLP_1024/CHEMBL2047_EC50/checkpoints/epoch=18-step=133.ckpt
Evaluating CHEMBL2047_EC50 (regression)


Predicting: |          | 0/? [00:00<?, ?it/s]

Evaluation completed for CHEMBL2047_EC50 (regression)


### pt_MLP_2048

In [11]:
pt_MLP_2048_results = evaluate_datasets(chembl_datasets, "pt_MLP_2048", fingerprint_length=2048)

/usr/local/lib/python3.10/dist-packages/lightning/pytorch/utilities/parsing.py:199: Attribute 'net' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['net'])`.
INFO: GPU available: False, used: False
INFO:lightning.pytorch.utilities.rank_zero:GPU available: False, used: False
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


Loaded checkpoint from src/models/pretrained/property_prediction/classification/pt_MLP_2048/CHEMBL234_Ki/checkpoints/epoch=4-step=185.ckpt
Evaluating CHEMBL234_Ki (classification)


Predicting: |          | 0/? [00:00<?, ?it/s]

Evaluation completed for CHEMBL234_Ki (classification)


/usr/local/lib/python3.10/dist-packages/lightning/pytorch/utilities/parsing.py:199: Attribute 'net' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['net'])`.
INFO: GPU available: False, used: False
INFO:lightning.pytorch.utilities.rank_zero:GPU available: False, used: False
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


...

Evaluation completed for CHEMBL2047_EC50 (classification)


/usr/local/lib/python3.10/dist-packages/lightning/pytorch/utilities/parsing.py:199: Attribute 'net' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['net'])`.
INFO: GPU available: False, used: False
INFO:lightning.pytorch.utilities.rank_zero:GPU available: False, used: False
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


Loaded checkpoint from src/models/pretrained/property_prediction/regression/pt_MLP_2048/CHEMBL2047_EC50/checkpoints/epoch=18-step=133.ckpt
Evaluating CHEMBL2047_EC50 (regression)


Predicting: |          | 0/? [00:00<?, ?it/s]

Evaluation completed for CHEMBL2047_EC50 (regression)


### pt_MLP_2048 + layernorm

In [12]:
pt_MLP_ln_2048_results = evaluate_datasets(chembl_datasets, "pt_MLP_ln_2048", fingerprint_length=2048)

/usr/local/lib/python3.10/dist-packages/lightning/pytorch/utilities/parsing.py:199: Attribute 'net' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['net'])`.
INFO: GPU available: False, used: False
INFO:lightning.pytorch.utilities.rank_zero:GPU available: False, used: False
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


Loaded checkpoint from src/models/pretrained/property_prediction/classification/pt_MLP_ln_2048/CHEMBL234_Ki/checkpoints/epoch=4-step=185.ckpt
Evaluating CHEMBL234_Ki (classification)


Predicting: |          | 0/? [00:00<?, ?it/s]

Evaluation completed for CHEMBL234_Ki (classification)


/usr/local/lib/python3.10/dist-packages/lightning/pytorch/utilities/parsing.py:199: Attribute 'net' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['net'])`.
INFO: GPU available: False, used: False
INFO:lightning.pytorch.utilities.rank_zero:GPU available: False, used: False
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


...

Evaluation completed for CHEMBL2047_EC50 (classification)


/usr/local/lib/python3.10/dist-packages/lightning/pytorch/utilities/parsing.py:199: Attribute 'net' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['net'])`.
INFO: GPU available: False, used: False
INFO:lightning.pytorch.utilities.rank_zero:GPU available: False, used: False
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


Loaded checkpoint from src/models/pretrained/property_prediction/regression/pt_MLP_ln_2048/CHEMBL2047_EC50/checkpoints/epoch=18-step=133.ckpt
Evaluating CHEMBL2047_EC50 (regression)


Predicting: |          | 0/? [00:00<?, ?it/s]

Evaluation completed for CHEMBL2047_EC50 (regression)


### ptMLP_recon_2048

In [13]:
pt_recon_MLP_2048_results = evaluate_datasets(chembl_datasets, "pt_recon_MLP_2048", fingerprint_length=2048)

/usr/local/lib/python3.10/dist-packages/lightning/pytorch/utilities/parsing.py:199: Attribute 'net' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['net'])`.
INFO: GPU available: False, used: False
INFO:lightning.pytorch.utilities.rank_zero:GPU available: False, used: False
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


Loaded checkpoint from src/models/pretrained/property_prediction/classification/pt_recon_MLP_2048/CHEMBL234_Ki/checkpoints/epoch=8-step=333.ckpt
Evaluating CHEMBL234_Ki (classification)


Predicting: |          | 0/? [00:00<?, ?it/s]

Evaluation completed for CHEMBL234_Ki (classification)


/usr/local/lib/python3.10/dist-packages/lightning/pytorch/utilities/parsing.py:199: Attribute 'net' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['net'])`.
INFO: GPU available: False, used: False
INFO:lightning.pytorch.utilities.rank_zero:GPU available: False, used: False
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


...

Evaluation completed for CHEMBL2047_EC50 (classification)


/usr/local/lib/python3.10/dist-packages/lightning/pytorch/utilities/parsing.py:199: Attribute 'net' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['net'])`.
INFO: GPU available: False, used: False
INFO:lightning.pytorch.utilities.rank_zero:GPU available: False, used: False
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


Loaded checkpoint from src/models/pretrained/property_prediction/regression/pt_recon_MLP_2048/CHEMBL2047_EC50/checkpoints/epoch=41-step=294.ckpt
Evaluating CHEMBL2047_EC50 (regression)


Predicting: |          | 0/? [00:00<?, ?it/s]

Evaluation completed for CHEMBL2047_EC50 (regression)


### ptMLP_recon_2048 + layernorm

In [14]:
pt_recon_MLP_ln_2048_results = evaluate_datasets(chembl_datasets, "pt_recon_MLP_ln_2048", fingerprint_length=2048)

/usr/local/lib/python3.10/dist-packages/lightning/pytorch/utilities/parsing.py:199: Attribute 'net' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['net'])`.
INFO: GPU available: False, used: False
INFO:lightning.pytorch.utilities.rank_zero:GPU available: False, used: False
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


Loaded checkpoint from src/models/pretrained/property_prediction/classification/pt_recon_MLP_ln_2048/CHEMBL234_Ki/checkpoints/epoch=14-step=555.ckpt
Evaluating CHEMBL234_Ki (classification)


Predicting: |          | 0/? [00:00<?, ?it/s]

Evaluation completed for CHEMBL234_Ki (classification)


/usr/local/lib/python3.10/dist-packages/lightning/pytorch/utilities/parsing.py:199: Attribute 'net' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['net'])`.
INFO: GPU available: False, used: False
INFO:lightning.pytorch.utilities.rank_zero:GPU available: False, used: False
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


...

Evaluation completed for CHEMBL2047_EC50 (classification)


/usr/local/lib/python3.10/dist-packages/lightning/pytorch/utilities/parsing.py:199: Attribute 'net' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['net'])`.
INFO: GPU available: False, used: False
INFO:lightning.pytorch.utilities.rank_zero:GPU available: False, used: False
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


Loaded checkpoint from src/models/pretrained/property_prediction/regression/pt_recon_MLP_ln_2048/CHEMBL2047_EC50/checkpoints/epoch=12-step=91.ckpt
Evaluating CHEMBL2047_EC50 (regression)


Predicting: |          | 0/? [00:00<?, ?it/s]

Evaluation completed for CHEMBL2047_EC50 (regression)


## Print to file

In [15]:
results_dict = {
    "MLP_2048": MLP_2048_results,
    "MLP_1024": MLP_1024_results,
    "MLP_256": MLP_256_results,
    "halfstepMLP_2048" : halfstepMLP_2048_results,
    "halfstepMLP_1024" : halfstepMLP_1024_results,
    "pt_MLP_2048" : pt_MLP_2048_results,
    "pt_MLP_ln_2048" : pt_MLP_ln_2048_results,
    "pt_recon_MLP_2048" : pt_recon_MLP_2048_results,
    "pt_recon_MLP_ln_2048" : pt_recon_MLP_ln_2048_results
    }

In [16]:
plot_data_dir = os.path.join("data", "plot")
if not os.path.exists(plot_data_dir):
    os.makedirs(plot_data_dir)

file_path = os.path.join(plot_data_dir, "mlp_results.csv")

write_results_to_csv(results_dict, file_path)

## Refs

[1] Derek van Tilborg, Alisa Alenicheva, and Francesca Grisoni.“Exposing the Limitations of Molecular Machine Learning with Activity Cliffs”. In: Journal of Chemical Information and Modeling 62.23 (Dec. 2022), pp. 5938–5951. DOI: 10.1021/acs.jcim.2c01073. URL: https://doi.
org/10.1021/acs.jcim.2c01073.   
[2] César Miguel Valdez Córdova. Towards learning activity cliff-aware molecular representations. Publication pending.